In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display

In [2]:
folder_path = "../data/halfhourly_dataset/halfhourly_dataset/"
print(os.getcwd())
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        try:
            # Try reading the CSV file with 'utf-8' encoding
            df = pd.read_csv(filepath, encoding='utf-8')
        except UnicodeDecodeError:
            # If 'utf-8' encoding fails, try 'latin-1'
            df = pd.read_csv(filepath, encoding='latin-1')
        dfs.append(df)
        
df = pd.concat(dfs, ignore_index=True)

/home/tim/code/machine_learning/final_project/Power_Forecasting_With_DNN/src


/tmp/ipykernel_32198/1353154252.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')
/tmp/ipykernel_32198/1353154252.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')
/tmp/ipykernel_32198/1353154252.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')
/tmp/ipykernel_32198/1353154252.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')
/tmp/ipykernel_32198/1353154252.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')
/tmp/ipykernel_32198/1353154252.py:9: DtypeWarning: Columns (2) have mixed 

In [3]:
df = df.rename(columns={"tstp": "timestamp"})
df.sort_values(by='timestamp', inplace=True)
display(df.head())

,LCLid,timestamp,energy(kWh/hh)
8960902,MAC000145,2011-11-23 09:00:00.0000000,0.355
152683783,MAC000146,2011-11-23 09:00:00.0000000,0.214
152683784,MAC000146,2011-11-23 09:30:00.0000000,0.147
8960903,MAC000145,2011-11-23 09:30:00.0000000,0.414
8960904,MAC000145,2011-11-23 10:00:00.0000000,0.335


In [6]:
from datetime import timedelta
import copy
weather_df = pd.read_csv("../data/weather_hourly_darksky.csv")
weather_df = weather_df.rename(columns={"time": "timestamp"})
weather_df = weather_df.drop('icon', axis=1)
weather_df = weather_df.sort_values(by='timestamp')
# change timestamp to datetime
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'], utc=True)
display(weather_df.head())
start_time = weather_df['timestamp'].iloc[0]
end_time = weather_df['timestamp'].iloc[-1]
iterated_time = start_time + timedelta(minutes=30)
all_needed_times = [copy.deepcopy(iterated_time)]
while iterated_time < end_time:
    iterated_time = iterated_time + timedelta(minutes=30)
    all_needed_times.append(copy.deepcopy(iterated_time))
time_df = pd.DataFrame({'timestamp': all_needed_times})
display(time_df.head())
result_df = pd.merge(time_df, weather_df, on='timestamp', how='left')
result_df.sort_values(by='timestamp', inplace=True)
display(result_df.head())
special_columns = ['summary', 'precipType']
for col in special_columns:
    result_df[col].fillna(method='ffill', inplace=True)
display(result_df.head())
special_columns.append('timestamp')
remaining_columns = list(set(result_df.columns) - set(special_columns))
for col in remaining_columns:
    result_df[col].interpolate(method='linear', inplace=True)
display(result_df.head())



,visibility,windBearing,temperature,timestamp,dewPoint,pressure,apparentTemperature,windSpeed,precipType,humidity,summary
312,13.63,160,13.49,2011-11-01 00:00:00+00:00,11.48,1008.14,13.49,3.11,rain,0.88,Clear
313,13.26,154,12.73,2011-11-01 01:00:00+00:00,11.58,1007.88,12.73,3.08,rain,0.93,Partly Cloudy
314,12.94,161,13.65,2011-11-01 02:00:00+00:00,12.14,1007.09,13.65,3.71,rain,0.91,Clear
315,12.99,170,14.13,2011-11-01 03:00:00+00:00,12.24,1006.50,14.13,3.95,rain,0.88,Partly Cloudy
316,12.92,180,14.17,2011-11-01 04:00:00+00:00,12.59,1006.14,14.17,3.97,rain,0.90,Partly Cloudy


,timestamp
0,2011-11-01 00:30:00+00:00
1,2011-11-01 01:00:00+00:00
2,2011-11-01 01:30:00+00:00
3,2011-11-01 02:00:00+00:00
4,2011-11-01 02:30:00+00:00


,timestamp,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,humidity,summary
0,2011-11-01 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-11-01 01:00:00+00:00,13.26,154.0,12.73,11.58,1007.88,12.73,3.08,rain,0.93,Partly Cloudy
2,2011-11-01 01:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-11-01 02:00:00+00:00,12.94,161.0,13.65,12.14,1007.09,13.65,3.71,rain,0.91,Clear
4,2011-11-01 02:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


/tmp/ipykernel_32198/3659373260.py:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result_df[col].fillna(method='ffill', inplace=True)


,timestamp,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,humidity,summary
0,2011-11-01 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-11-01 01:00:00+00:00,13.26,154.0,12.73,11.58,1007.88,12.73,3.08,rain,0.93,Partly Cloudy
2,2011-11-01 01:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rain,NaN,Partly Cloudy
3,2011-11-01 02:00:00+00:00,12.94,161.0,13.65,12.14,1007.09,13.65,3.71,rain,0.91,Clear
4,2011-11-01 02:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rain,NaN,Clear


,timestamp,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,humidity,summary
0,2011-11-01 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-11-01 01:00:00+00:00,13.260,154.0,12.73,11.58,1007.880,12.73,3.080,rain,0.930,Partly Cloudy
2,2011-11-01 01:30:00+00:00,13.100,157.5,13.19,11.86,1007.485,13.19,3.395,rain,0.920,Partly Cloudy
3,2011-11-01 02:00:00+00:00,12.940,161.0,13.65,12.14,1007.090,13.65,3.710,rain,0.910,Clear
4,2011-11-01 02:30:00+00:00,12.965,165.5,13.89,12.19,1006.795,13.89,3.830,rain,0.895,Clear
